In [1]:
import snap
from operator import itemgetter
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from random import choice
from pandas import datetools
import datetime
import numpy as np

In [6]:
#GCC_filtered = nx.read_edgelist("data/edges_filtered.csv", delimiter='\t')
#edges_filtered = pd.read_csv('data/edges.csv',',')
#G = nx.read_edgelist("data/vent.edgelist")
#GCC = nx.read_edgelist("data/GCCEdges.csv")
#nodes = nx.read_edgelist("data/nodes_reduced.csv")
edges = pd.read_csv("data/edges_filtered.csv", ',')
metadata = pd.read_csv('data/metadata_filtered.csv',' ')
metadata['Timestamp']=metadata['created_at'].astype('datetime64')
#metadata=metadata.sort_values(by=['Timestamp'])

In [9]:
G = nx.from_pandas_edgelist(edges, '0', '1')

In [7]:
metadata.head()

,Unnamed: 0,emotion_id,user_id,created_at,reactions,emotion_category_id,emotion_name,enabled,category_name,Timestamp
0,19,134,3,2013-12-31 17:51:43.000,0,15,Sad,True,Sadness,2013-12-31 17:51:43
1,20,134,3,2014-01-01 12:34:33.000,0,15,Sad,True,Sadness,2014-01-01 12:34:33
2,21,134,3,2014-01-04 04:47:04.000,0,15,Sad,True,Sadness,2014-01-04 04:47:04
3,22,134,3,2014-01-04 06:01:53.000,0,15,Sad,True,Sadness,2014-01-04 06:01:53
4,23,134,3,2014-01-05 03:33:47.000,0,15,Sad,True,Sadness,2014-01-05 03:33:47


## Get the Ego-Network for the nodes with the highest degrees

In [10]:
node_and_degree = G.degree()

In [20]:
for ego in [1, 2, 3, 4]:
    
    (largest_hub, degree) = sorted(node_and_degree, key=itemgetter(1))[-ego]
    hub_ego = nx.ego_graph(G, largest_hub)
    hub_ego.remove_node(largest_hub)

    ### Creates a dataset with the edgelist of the ego-network (csv)
    ego_pd = nx.to_pandas_edgelist(hub_ego,'Source','Target')
    ego_pd.to_csv('data/ego_edges.csv', sep=',', index = False)
    ego_pd.head()

    ### Creates a dataset with the nodes in the ego-network (csv)
    ego_nodes = pd.concat([ego_pd['Source'], ego_pd['Target']]).unique()
    ego_nodes = pd.DataFrame(ego_nodes, columns=['id'])
    ego_nodes.to_csv('data/Ego '+str(ego)+'/ego_nodes'+str(ego)+'.csv', sep=',', index = False)
    ego_nodes.head()

    metadata['user_id']=metadata['user_id'].astype(int)
    ego_nodes['id']=ego_nodes['id'].astype(int)

    ### Ceate a metadata with the ids in the ego-network (csv)
    metadata[metadata['user_id'].isin(ego_nodes['id'])].user_id.nunique()
    ego_nodes.id.nunique()

    ego_pd_meta = metadata[metadata['user_id'].isin(ego_nodes['id'])]
    ego_pd_meta.head()

    ego_pd_meta=ego_pd_meta.rename(index=str, columns={"user_id": "id"})

    ego_pd_meta.to_csv('data/Ego '+str(ego)+'/ego_meta'+str(ego)+'.csv', sep=',', index = False)
    ego_pd.to_csv('data/Ego '+str(ego)+'/ego_edges'+str(ego)+'.csv', sep=',', index = False)
    
    ### Make another metadata csv with only the last vent of each id (only for Gephi)
    time = max(ego_pd_meta['Timestamp'])
    ego_pd_meta_last = ego_pd_meta[ego_pd_meta['Timestamp']<=time].sort_values('Timestamp').groupby('id').tail(1)
    ego_pd_meta_last.to_csv('data/Ego '+str(ego)+'/ego_meta_last'+str(ego)+'.csv', sep=',', index = False)